# Stats Logging

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
#export
import sys
sys.path.insert(0, '/'.join(sys.path[0].split('/')[:-1] + ['scripts']))

from learner import *

### Now that we made sure training is working, time to look at the metrics

In [3]:
#export
class AvgStats():
    '''Class for keeping track of average loss and other methods'''
    def __init__(self, metrics, training): 
        self.metrics, self.training = metrics, training
    
    def reset(self):
        self.count = 0
        self.total_loss = torch.Tensor([0])
        self.totals = [torch.Tensor([0])] * len(self.metrics)
        
    @property
    def all_stats(self): return [self.total_loss] + self.totals
    
    @property
    def avg_stats(self): return [s.item()/self.count for s in self.all_stats]
    
    def __repr__(self):
        if not self.count: return ''
        return f"{'train' if self.training else 'valid'} metrics - {self.avg_stats}"

    def accumulate(self, learner):
        batch_size = learner.x_batch.shape[0]
        self.count += batch_size
        self.total_loss = learner.loss * batch_size
        for i, metric in enumerate(self.metrics):
            self.totals[i] += metric(learner.pred, learner.y_batch) * batch_size

class StatsLogging(Callback):
    '''Callback for print out loss and other metrics for each training epoch'''
    def __init__(self, metrics=[compute_accuracy]):
        self.train_stats = AvgStats(metrics, True)
        self.valid_stats = AvgStats(metrics, False)
        
    def before_epoch(self):
        self.train_stats.reset()
        self.valid_stats.reset()
        
    def after_loss(self):
        stats = self.train_stats if self.model.training else self.valid_stats
        stats.accumulate(self.learner)
    
    def after_epoch(self):
        print(f'Epoch - {self.epoch}\n{self.train_stats}\n{self.valid_stats}\n')

# Tests

In [4]:
data_bunch = get_data_bunch(*get_mnist_data(), batch_size=64)
model = get_lin_model(data_bunch)
optimizer = DynamicOpt(list(model.parameters()), learning_rate=0.1)
loss_fn = CrossEntropy()
callbacks = [StatsLogging()]
learner = Learner(data_bunch, model, loss_fn, optimizer, callbacks)
print(learner)

(DataBunch) 
    (DataLoader) 
        (Dataset) x: (50000, 784), y: (50000,)
        (Sampler) total: 50000, batch_size: 64, shuffle: True
    (DataLoader) 
        (Dataset) x: (10000, 784), y: (10000,)
        (Sampler) total: 10000, batch_size: 128, shuffle: False
(Model)
    Linear(784, 50)
    ReLU()
    Linear(50, 10)
(CrossEntropy)
(DynamicOpt) hyper_params: ['learning_rate']
(Callbacks) ['TrainEval', 'StatsLogging']


In [5]:
learner.fit(3)

Epoch - 1
train metrics - [8.092031478881836e-05, 0.91324]
valid metrics - [3.977417945861816e-05, 0.9511]

Epoch - 2
train metrics - [2.8432035446166992e-05, 0.95594]
valid metrics - [1.3927602767944337e-05, 0.9606]

Epoch - 3
train metrics - [1.0415992736816407e-05, 0.96658]
valid metrics - [1.2269687652587891e-05, 0.9659]



In [6]:
x_train, y_train, x_valid, y_valid = get_mnist_data()
x_train, y_train, x_valid, y_valid = x_train[:8000], y_train[:8000], x_valid[:2000], y_valid[:2000]

data_bunch = get_data_bunch(x_train, y_train, x_valid, y_valid, batch_size=64)
model = get_conv_model(data_bunch)
optimizer = DynamicOpt(list(model.parameters()), learning_rate=0.1) # dynamic optimizer
loss_fn = CrossEntropy()
callbacks = [StatsLogging()]
learner = Learner(data_bunch, model, loss_fn, optimizer, callbacks)
print(learner)

(DataBunch) 
    (DataLoader) 
        (Dataset) x: (8000, 784), y: (8000,)
        (Sampler) total: 8000, batch_size: 64, shuffle: True
    (DataLoader) 
        (Dataset) x: (2000, 784), y: (2000,)
        (Sampler) total: 2000, batch_size: 128, shuffle: False
(Model)
    Reshape(1, 28, 28)
    Conv(1, 8, 5, 4)
    ReLU()
    Conv(8, 16, 3, 2)
    Flatten()
    Linear(256, 10)
(CrossEntropy)
(DynamicOpt) hyper_params: ['learning_rate']
(Callbacks) ['TrainEval', 'StatsLogging']


In [7]:
learner.fit(3)

Epoch - 1
train metrics - [0.0021502406597137453, 0.81175]
valid metrics - [0.024427162170410157, 0.883]

Epoch - 2
train metrics - [0.0025861341953277587, 0.908625]
valid metrics - [0.015295360565185547, 0.9115]

Epoch - 3
train metrics - [0.00155214524269104, 0.930125]
valid metrics - [0.018342151641845703, 0.918]



In [8]:
x_train, y_train, x_valid, y_valid = get_mnist_data()
x_train, y_train, x_valid, y_valid = x_train[:8000], y_train[:8000], x_valid[:2000], y_valid[:2000]

data_bunch = get_data_bunch(x_train, y_train, x_valid, y_valid, batch_size=64)
model = get_conv_final_model(data_bunch)
optimizer = DynamicOpt(list(model.parameters()), learning_rate=0.1)
loss_fn = CrossEntropy()
callbacks = [StatsLogging()]
learner = Learner(data_bunch, model, loss_fn, optimizer, callbacks)
print(learner)

(DataBunch) 
    (DataLoader) 
        (Dataset) x: (8000, 784), y: (8000,)
        (Sampler) total: 8000, batch_size: 64, shuffle: True
    (DataLoader) 
        (Dataset) x: (2000, 784), y: (2000,)
        (Sampler) total: 2000, batch_size: 128, shuffle: False
(Model)
    Reshape(1, 28, 28)
    Conv(1, 4, 5, 2)
    AvgPool(2, 1)
    BatchNorm()
    Conv(4, 16, 3, 2)
    BatchNorm()
    Flatten()
    Linear(400, 64)
    ReLU()
    Linear(64, 10)
(CrossEntropy)
(DynamicOpt) hyper_params: ['learning_rate']
(Callbacks) ['TrainEval', 'StatsLogging']


In [9]:
learner.fit(3)

Epoch - 1
train metrics - [0.0038621110916137696, 0.8345]
valid metrics - [0.020593032836914063, 0.8885]

Epoch - 2
train metrics - [0.0014318745136260985, 0.916125]
valid metrics - [0.016359914779663087, 0.911]

Epoch - 3
train metrics - [0.001374750018119812, 0.935875]
valid metrics - [0.011205047607421874, 0.923]

